# Model building and prediction

In [1]:
import pandas as pd
import numpy as np
import os
import cv2

In [2]:
# image center 50*50 piexls data extraction
def ave_rgb(img):
    x,y,_ = img.shape
    temp = []
    for i in range(x//2-25,x//2+25):
        for j in range(y//2-25,y//2+25):
            rgb = img[i,j]
            temp.append(rgb.tolist())
    df_temp = pd.DataFrame(temp)
    return df_temp.mean().values.tolist()

In [3]:
# Calculat the average RGB values of 50*50 pixels and store in the data list files
# Data list is the excel file that store all the image information, including image number, concentration, features
import math

temp = os.listdir("./working folder")
dir = './working folder'
name_list = []
for ele in temp:
    if ('JPG' in ele) or ('jpg' in ele) :
        name_list.append(ele)
df = pd.read_excel('./working folder/data list.xlsx')
reshap_res = []
for name in name_list:
    img = cv2.imread(os.path.join(dir,name))
    r,g,b = ave_rgb(img)
    num = int(name.split('.')[0])
    row = df[df['No']==num].values.tolist()[0]
    reshap_res.append(row+[math.trunc(r),math.trunc(g),math.trunc(b)])

In [5]:
dfsn = pd.DataFrame(reshap_res,columns = ['Label','Con','Abs','No','container','light','phone','R','G','B'])

from sklearn.model_selection import train_test_split
inputs = dfsn[['vessel','light','phone','R','G','B','Con']]

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd

# Assuming dfsn is your DataFrame
inputs = dfsn.loc[(((dfsn['No'] < 1922) & (dfsn['No'] > 0))),:][['container', 'light', 'Label', 'phone', 'R', 'G', 'B', 'Con']]


# Lists to store RMSE, R2, and feature importances for each split
rmses = []
r2s = []
feature_importances = []

# Loop over different random states for train-test split
for random_state in range(42, 62):
    train, test = train_test_split(inputs, test_size=0.2, stratify=inputs['Label'], random_state=random_state)

    train_y = train['Con']
    train_x = train.drop(['Label', 'Con'], axis=1)

    test_y = test['Con']
    test_x = test.drop(['Label', 'Con'], axis=1)

    # Train the model
    model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=random_state)
    model.fit(train_x, train_y)

    # Store the feature importances
    feature_importances.append(model.feature_importances_)

    # Make predictions
    pre_test_y = model.predict(test_x)

    # Calculate and store RMSE and R2
    rmse = np.sqrt(mean_squared_error(test_y, pre_test_y))
    r2 = r2_score(test_y, pre_test_y)
    rmses.append(rmse)
    r2s.append(r2)

# Calculate average and standard deviation of RMSE, R2, and feature importances
average_rmse = np.mean(rmses)
std_rmse = np.std(rmses)
average_r2 = np.mean(r2s)
std_r2 = np.std(r2s)

# Convert list of feature importances to DataFrame and calculate mean and std
feature_importances_df = pd.DataFrame(feature_importances, columns=train_x.columns)
average_feature_importances = feature_importances_df.mean()
std_feature_importances = feature_importances_df.std()

print(f"Average RMSE: {average_rmse}, Standard Deviation of RMSE: {std_rmse}")
print(f"Average R2: {average_r2}, Standard Deviation of R2: {std_r2}")
print("Average Feature Importances:")
print(average_feature_importances)
print("\nStandard Deviation of Feature Importances:")
print(std_feature_importances)
